<a href="https://colab.research.google.com/github/massone99/visione_artificiale_colab_notebooks/blob/main/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Esercitazione su image classification**
Nell'esercitazione odierna utilizzeremo una *Convolutional Neural Network* (CNN) per applicazioni di riconoscimento di volti (*Face Recognition*).

Faremo uso del framework **TensorFlow**, sfruttando la libreria open-source **Keras** appositamente progettata per permettere una rapida prototipazione di reti neurali profonde.

Alcuni link di approfondimento:
- Introduzione a TensorFlow con utile schema grafico delle [API disponibili](https://ekababisong.org/gcp-ml-seminar/tensorflow/#navigating-through-the-tensorflow-api)
- [Keras](https://keras.io/)

Nello specifico sarà utilizzata la rete [VGG-Face](https://www.robots.ox.ac.uk/~vgg/publications/2015/Parkhi15/parkhi15.pdf) pre-addestrata sul [dataset VGG-Face](https://www.robots.ox.ac.uk/~vgg/data/vgg_face/) (contenente oltre 2 milioni di immagini di volti appartenenti a più di 2000 soggetti).

L'obiettivo dell'esercitazione è quello di utilizzare una CNN pre-addestrata come *feature extractor* per il riconoscimento di volti.

# **Operazioni preliminari**
Prima di incominciare, è necessario eseguire alcune operazioni preliminari.

Eseguendo la cella sottostante tutto il materiale necessario per lo svolgimento dell'esercitazione verrà scaricato sulla macchina remota. Alla fine dell'esecuzione selezionare il tab **Files** per verificare che tutto sia stato scaricato correttamente.


In [ ]:
!wget http://bias.csr.unibo.it/VR/Esercitazioni/DBs/ImageClassification/FaceScrubSubset_Celebrities.zip
!wget http://bias.csr.unibo.it/VR/Esercitazioni/MaterialeEsImageClassification.zip
!wget http://bias.csr.unibo.it/VR/Esercitazioni/PythonUtilities.zip

!unzip -q /content/FaceScrubSubset_Celebrities.zip
!unzip -q /content/MaterialeEsImageClassification.zip
!unzip -q /content/PythonUtilities.zip

!rm /content/FaceScrubSubset_Celebrities.zip
!rm /content/MaterialeEsImageClassification.zip
!rm /content/PythonUtilities.zip

# **Import delle librerie**
Per prima cosa è necessario eseguire l'import delle librerie utilizzate durante l'esecitazione.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import cv2
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.decomposition import PCA

import vr_utilities

# **Dataset**
Il dataset ultilizzato è composto da immagini RGB di volti di persone famose. In particolare utilizzeremo un sottoinsieme del [FaceScrub](http://vintage.winklerbros.net/facescrub.html) contenente 1590 immagini di 530 soggetti diversi (3  immagini per ciascuno di essi, 2 per il training e 1 per il test).

Visto il numero esiguo di immagini (1060 per il dataset di training), cercare di addestrare da zero una CNN complessa (partendo da pesi random) risulta impossibile.

In [ ]:
db_path = '/content/Celebrities'
train_filelist = 'TrainingSet.txt'
test_filelist = 'TestSet.txt'
labelnames_list = 'LabelNames.txt'

print('Caricamento in corso ...')
or_train_x, train_y = vr_utilities.load_labeled_dataset(train_filelist, db_path)
or_test_x, test_y = vr_utilities.load_labeled_dataset(test_filelist, db_path)

label_names = vr_utilities.load_label_names(labelnames_list, db_path)

print('Shape training set:', or_train_x.shape)
print('Shape test set:', or_test_x.shape)

La cella seguente contiene il codice per mostrare alcune immagini del training set. Guardando alcuni esempi si può facilmente notare la grande variabilità in termini di:
- posa;
- illuminazione;
- espressione.

In [ ]:
rows = 3
columns = 6

plt.rcParams.update({'font.size': 12})
_, axs = plt.subplots(rows, columns, squeeze=False,figsize=(20, 10))
samples = random.sample(range(len(label_names)), columns)

for j in range(columns):
    idx=samples[j]
    sel_train_images=[or_train_x[k] for k in np.where(train_y==idx)[0]]
    sel_test_images=[or_test_x[k] for k in np.where(test_y==idx)[0]]
    sel_images=sel_train_images+sel_test_images
    axs[0, j].set_title(label_names[idx])
    for i in range(rows):
        axs[i, j].axis('off')
        axs[i, j].imshow(sel_images[i])

# **VGG-Face**
*VGG-Face* è una CNN introdotta per applicazioni di riconoscimento del volto.

È composta da:
- 16 *layer* di **convoluzione**;
- 5 *layer* di **max pooling**.

<img src=https://biolab.csr.unibo.it/vr/esercitazioni/NotebookImages/EsImageClassification\vgg-face-architecture.jpg width="1000">


## **Definizione del modello**
La funzione seguente crea il modello della *VGG-Face*.

In [ ]:
def build_vggface():
  model=keras.Sequential(
          [
              layers.Input(shape=(224,224,3),name='input'),
              layers.Conv2D(filters=64, kernel_size=3,padding='same', activation='relu',name='conv1_1-relu1_1'),
              layers.Conv2D(filters=64, kernel_size=3,padding='same', activation='relu',name='conv1_2-relu1_2'),
              layers.MaxPooling2D(pool_size=2, strides=2,name='pool1'),
              layers.Conv2D(filters=128, kernel_size=3,padding='same', activation='relu',name='conv2_1-relu2_1'),
              layers.Conv2D(filters=128, kernel_size=3,padding='same', activation='relu',name='conv2_2-relu2_2'),
              layers.MaxPooling2D(pool_size=2, strides=2,name='pool2'),
              layers.Conv2D(filters=256, kernel_size=3,padding='same', activation='relu',name='conv3_1-relu3_1'),
              layers.Conv2D(filters=256, kernel_size=3,padding='same', activation='relu',name='conv3_2-relu3_2'),
              layers.Conv2D(filters=256, kernel_size=3,padding='same', activation='relu',name='conv3_3-relu3_3'),
              layers.MaxPooling2D(pool_size=2, strides=2,name='pool3'),
              layers.Conv2D(filters=512, kernel_size=3,padding='same', activation='relu',name='conv4_1-relu4_1'),
              layers.Conv2D(filters=512, kernel_size=3,padding='same', activation='relu',name='conv4_2-relu4_2'),
              layers.Conv2D(filters=512, kernel_size=3,padding='same', activation='relu',name='conv4_3-relu4_3'),
              layers.MaxPooling2D(pool_size=2, strides=2,name='pool4'),
              layers.Conv2D(filters=512, kernel_size=3,padding='same', activation='relu',name='conv5_1-relu5_1'),
              layers.Conv2D(filters=512, kernel_size=3,padding='same', activation='relu',name='conv5_2-relu5_2'),
              layers.Conv2D(filters=512, kernel_size=3,padding='same', activation='relu',name='conv5_3-relu5_3'),
              layers.MaxPooling2D(pool_size=2, strides=2,name='pool5'),
              layers.Conv2D(filters=4096, kernel_size=7, activation='relu',name='fc6-relu6'),
              layers.Dropout(0.5,name='do6'),
              layers.Conv2D(filters=4096, kernel_size=1, activation='relu',name='fc7-relu7'),
              layers.Dropout(0.5,name='do7'),
              layers.Conv2D(filters=2622, kernel_size=1,activation='softmax',name='fc8-prob'),
              layers.Flatten(name='flatten'),
          ]
        )

  return model

## **Creazione del modello**
Il codice seguente crea una *VGG-Face* richiamando la funzione **build_vggface** definita sopra.

In [ ]:
model=build_vggface()

## **Visualizzazione del modello**
Eseguendo la cella seguente è possibile stampare un riepilogo testuale della struttura della rete.

In [ ]:
model.summary()

Se si preferisce una visualizzazione grafica, eseguire la cella seguente.

In [ ]:
keras.utils.plot_model(model,show_shapes=True, show_layer_names=True)

## **Caricamento dei pesi**
I pesi della rete *VGG-Face* addestrata sul dataset VGG-Face sono stati resi disponibili e possono essere caricati senza dover ripetere l'addestramento.

Tramite il metodo **load_weights** è possibile caricare all'interno del modello i pesi memorizzati all'interno di un file.

In [ ]:
model.load_weights('vgg_face_weights.h5')

# **Creazione dell'estrattore di feature**
Il dataset che utilizzeremo nell'esercitazione contiene soggetti diversi rispetto a quelli utilizzati per addestrare la VGG-Face.

L'addestramento di una CNN su un nuovo problema, richiede un hardware sufficientemente potente e un training set etichettato di notevoli dimensioni.

In alternativa al training da zero, possiamo utilizzare una rete esistente (pre-trained) per estrarre le feature generate ai livelli intermedi durante il passo forward ([*Transfer Learning*](https://cs231n.github.io/transfer-learning/)).

<img src=https://biolab.csr.unibo.it/vr/esercitazioni/NotebookImages/EsImageClassification\TransferLearning.png width="600">

Le feature possono essere utilizzate per:
1. addestrare un classificatore esterno (es. SVM) a
riconoscere i pattern del nuovo dominio applicativo;
2. stimare il grado di similarità tra feature estratte da immagini differenti utilizzando una metrica (es. distanza euclidea o distanza coseno).

L'operazione di estrazione delle feature consiste nel calcolare, per ogni immagine fornita in input, l'output della rete al livello desiderato (*layer_name*).

Per evitare, durante il passo *forward*, di attraversare livelli non necessari della rete si può creare una nuova istanza della classe [**Model**](https://keras.io/api/models/model/) il cui input sara il medesimo del modello originale mentre l'ouput sarà rappresentato dal livello da cui si vogliono estrarre le feature (*layer_name*).

In [ ]:
layer_name='fc6-relu6'
#layer_name='fc7-relu7'

feature_extractor = keras.Model(inputs=model.input,outputs=model.get_layer(layer_name).output)

print('Inputs: %s' % feature_extractor.inputs)
print('Outputs: %s' % feature_extractor.outputs)

## **Visualizzazione del nuovo modello**
Eseguendo la cella seguente è possibile stampare un riepilogo testuale della struttura della rete da utilizzare come feature extractor.

In [ ]:
feature_extractor.summary()

Se si preferisce una visualizzazione grafica, eseguire la cella seguente.

In [ ]:
keras.utils.plot_model(feature_extractor,show_shapes=True, show_layer_names=True)

# ***Pre-processing* delle immagini**
Il modello di VGG-Face di cui abbiamo caricato i pesi è stato addestrato con delle immagini pre-elaborate. Sarà necessario eseguire le medesime operazioni sia sul training che sul test set prima di poterli utilizzare.

Le immagini del nostro dataset presentano un'intensità luminosa nel range [0;1]. Per poter essere utilizzate con la rete pre-addestrata dovremo preventivamente "mappare" l'intensità nel range [-1;1]. Si esegua la cella seguente per effettuare tale *mapping*.

In [ ]:
print('Range originale: [',np.min(or_train_x),';',np.max(or_train_x),']')

norm_train_x=(or_train_x*2)-1
norm_test_x=(or_test_x*2)-1

print('Range ri-mappato: [',np.min(norm_train_x),';',np.max(norm_train_x),']')

# **Estrazione delle feature**
Per estrarre le feature è sufficiente richiamare il metodo [**predict(...)**](https://keras.io/api/models/model_training_apis/#predict-method) del nostro estrattore (*feature_extractor*).

Eseguire la cella seguente per estrarre le feature dal training e dal test set.

In [ ]:
print('Estrazione delle feature...')
train_features_x=feature_extractor.predict(norm_train_x)
test_features_x=feature_extractor.predict(norm_test_x)

print('Shape ndarray delle feature di train: ', train_features_x.shape)
print('Shape ndarray delle feature di test: ', test_features_x.shape)

Per comodità, può essere utile rimuovere le dimensioni unitarie tramite la funzione [**squeeze(...)**](https://numpy.org/doc/stable/reference/generated/numpy.squeeze.html) di NumPy.

In [ ]:
train_features_x=np.squeeze(train_features_x)
test_features_x=np.squeeze(test_features_x)

print('Shape ndarray delle feature di train: ', train_features_x.shape)
print('Shape ndarray delle feature di test: ', test_features_x.shape)

# **Face Recognition**
Le feature appena estratte possono essere direttamente utilizzate insieme alla [distanza coseno](https://en.wikipedia.org/wiki/Cosine_similarity) per effettuare *face recognition* sulle nostre immagini.

Dati due vettori **a** e **b**, la distanza coseno può essere calcolata come:

\begin{align}
D_C(\mathbf{a},\mathbf{b})=1-\frac{\mathbf{a} \cdot{} \mathbf{b}}{\lVert \mathbf{a} \rVert \lVert \mathbf{b} \rVert}
\end{align}

La funzione **compute_cosine_distances(...)**, definita nella cella seguente, calcola le distanze coseno delle feature di una immagine di test (*query_features_x*) da tutte le feature del training set (*train_features_x*). Questa implementazione permette di calcolare la norma di ogni immagine di test una sola volta.

In [ ]:
def compute_cosine_distances(train_features_x,query_features_x):
  cosine_distances=[]
  norm_query=np.linalg.norm(query_features_x)
  for train_feature in train_features_x:
    norm_train=np.linalg.norm(train_feature)
    cos_dist=1-np.dot(query_features_x, train_feature)/(norm_query*norm_train)
    cosine_distances.append(cos_dist)

  return np.asarray(cosine_distances)

## **Test**
La cella sottostante calcola tutte le distanze coseno tra le feature del dataset di test e quelle di training memorizzandole nella variabile *test_distances*.

In [ ]:
test_distances=[]
print('Calcolo distanze coseno ...')
for test_features in test_features_x:
  test_distances.append(compute_cosine_distances(train_features_x,test_features))

test_distances=np.asarray(test_distances)

print('Shape ndarray delle distanze: ', test_distances.shape)

È possibile misurare l'accuratezza del sistema di *face recognition* implementato eseguendo la cella successiva.

In [ ]:
test_distances_sorted_indices=np.argsort(test_distances,axis=1)

predicted_y=train_y[test_distances_sorted_indices[:,0]]

errors = predicted_y != test_y

accuracy=1-(errors.sum()/len(errors))
print('Accuracy sul test set: %.3f' % (accuracy))

## **Visualizzazione errori**
La cella seguente permette di visualizzare le immagini di test che vengono classificate in maniera errata. Sopra ad ogni immagine è riportato il nome del soggetto mentre a lato le classi più probabili.

In [ ]:
error_indices = np.where(errors == True)[0]

if error_indices.shape[0] > 0:
  # Visualizzazione immagini
  image_per_row = 2
  top_class_count = 5

  row_count=math.ceil(len(error_indices)/image_per_row)
  column_count=image_per_row
  plt.rcParams.update({'font.size': 12})
  _, axs = plt.subplots(row_count, column_count,figsize=(20, 4*row_count),squeeze=False)

  for i in range(row_count):
    for j in range(column_count):
      axs[i,j].axis('off')

  for i in range(len(error_indices)):
    q = i // image_per_row
    r = i % image_per_row
    idx = error_indices[i]

    axs[q,r].imshow(or_test_x[idx])
    axs[q,r].set_title(label_names[test_y[idx]])

    best_indices=test_distances_sorted_indices[idx,0:2*top_class_count]
    best_distances=test_distances[idx,best_indices]

    best_y = train_y[best_indices]
    _, unique_indices = np.unique(best_y, return_index=True)
    unique_indices=np.sort(unique_indices)

    text=''
    for j in range(top_class_count):
        text+='{}: {:.3f}\n'.format(label_names[best_y[unique_indices[j]]],best_distances[unique_indices[j]])

    axs[q,r].text(330, 150, text, horizontalalignment='left', verticalalignment='center')

# **Esercizio 1**
Utilizzare il sistema implementato per verificare a quale tra le celebrità presenti nel dataset assomigliate maggiormente.

A tal fine:

1. scattare una foto con il proprio volto in primo piano;
2. ritagliarla per ottenere un'immagine quadrata (rapporto 1:1);
3. riscalare l'immagine a una dimensione 224 x 224 pixel;
4. trasferire l'immagine ottenuta su **Colab** utilizzando la funzione *Upload* del tab **Files**;
5. caricare l'immagine in una variabile (per farlo può essere utile la funzione [**imread(...)**](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imread.html) della libreria [**Matplotlib**](https://matplotlib.org/));
6. effettuare il *pre-processing* dell'immagine;
7. estrarre le feature utilizzando l'estrattore creato;
8. calcolare la distanza coseno tra le feature estratte e quelle del training set;
9. visualizzare, utilizzando la libreria Matplotlib, il nome e le foto delle 3 celebrità più somiglianti.

In [ ]:
image_file_path = #...
similar_count = 3

# Punto 5
my_image = #...

# Punto 6
my_norm_image = #...

# Punto 7
my_image_features= #...
my_image_features=np.squeeze(my_image_features)

# Punto 8
my_distances= #...

# Punto 9
my_distances_sorted_indices=np.argsort(my_distances)
best_indices=my_distances_sorted_indices[0:similar_count]
best_distances=my_distances[best_indices]

plt.imshow((my_image))
plt.axis('off')

best_y = train_y[best_indices]

text=''
for j in range(similar_count):
    text+='{}: {:.3f}\n'.format(label_names[best_y[j]],best_distances[j])

plt.text(my_image.shape[1]*1.05, my_image.shape[0]*0.5, text, horizontalalignment='left', verticalalignment='center')

similar_face_indices=[i for i in best_indices if train_y[i] in best_y[:similar_count]]

plt.rcParams.update({'font.size': 13})
_, axs = plt.subplots(1, len(similar_face_indices),figsize=(20, 4))

for i in range(len(similar_face_indices)):
    idx=similar_face_indices[i]
    axs[i].axis('off')
    axs[i].imshow(or_train_x[idx])
    axs[i].set_title(label_names[train_y[idx]])

# **Esercizio 2**
Ripetere l'esercizio 1 utilizzando le feature restituite dal layer *fc7-relu7* anziché quelle del layer *fc6-relu6*.

# **Esercizio 3**
Addestrare un classificatore (es. SVM) a riconoscere le classi del nostro dataset.

Per svolgere l'esercizio si suggerisce di:
1. per ridurre i tempi di calcolo, ridurre la dimensionalità dei vettori di feature dei dataset di training e test. Per farlo si consiglia di utilizzare la classe [**PCA**](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) messa a disposizione dalla libreria **Scikit-learn** e in particolare i metodi [**fit**](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA.fit) e [**transform**](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA.transform);
2. creare un classificatore utilizzando le classi messe a disposizione da **OpenCV** (si veda esercitazione introduttiva);
3. addestrare il classificatore passando le feature di training (a dimensionalità ridotta) al metodo **train**;
4. valutare le prestazioni del classificatore appena addestrato utilizzando il metodo **predict** sul dataset di test (a dimensionalità ridotta);
5. ridurre la dimensionalità delle feature estratte dalla foto utilizzata nell'Esercizio 1 utilizzando la **PCA** del punto 1;
6. predire la classe di appartenenza più probabile delle feature estratte dalla foto utilizzata nell'Esercizio 1;
7. visualizzare, utilizzando la libreria Matplotlib, il nome e le foto della celebrità a voi più somigliante.


In [ ]:
# Punto 1
print('Shape delle feature di train: ', train_features_x.shape)
print('Shape delle feature di test: ', test_features_x.shape)
pca = #...
#...
red_train_features_x= #...
red_test_features_x= #...
print('Shape delle feature di train ridotte: ', red_train_features_x.shape)
print('Shape delle feature di test ridotte: ', red_test_features_x.shape)

# Punto 2
classifier=#...
#...

# Punto 3
#...

# Punto 4
_,pred_y= #...
pred_y=np.squeeze(pred_y)
errors = pred_y != test_y
accuracy=1-(errors.sum()/len(errors))
print('Accuracy sul test set: %.3f' % (accuracy))

# Punto 5
red_my_image_features= #...

# Punto 6
_,my_pred_y= #...

# Punto 7
my_pred_y=np.int32(my_pred_y)[0]
pred_class_train_pos=np.where(train_y==my_pred_y)[0]
pred_class_test_pos=np.where(test_y==my_pred_y)[0]

_, axs = plt.subplots(1, 4,figsize=(20, 4))

axs[0].axis('off')
axs[0].imshow(my_image)

axs[1].axis('off')
axs[1].imshow(or_train_x[pred_class_train_pos[0]])
axs[1].set_title(label_names[my_pred_y][0])

axs[2].axis('off')
axs[2].imshow(or_train_x[pred_class_train_pos[1]])
axs[2].set_title(label_names[my_pred_y][0])

axs[3].axis('off')
axs[3].imshow(or_test_x[pred_class_test_pos[0]])
axs[3].set_title(label_names[my_pred_y][0])